# DATA CLEANING

## Introduction

In this project, we will walk through the full data science life cycle, from data cleaning and feature selection to machine learning. We will focus on credit modelling, a well known data science problem that focuses on modeling a borrower's credit risk.

Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return.

The data can be obtained from [here](https://www.kaggle.com/wordsforthewise/lending-club/data?select=rejected_2007_to_2018Q4.csv.gz). The data was initially about 1.7 GB and covering a period of 2007 to 2018. We divided it into four parts and only selected data from 2014 to 2018. The data had 151 columns, so we decided to drop those columns containing many missing values- about over 90% missing values. Since these computations were time consuming we had to edit those out to avoid rerunning them. We saved our new data set having 53 columns as loans.csv

We will start by cleaning up the loans.csv, prepare some features and then make some predictions.

For information about the columns please follow [this link](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097).

In [1]:
import pandas as pd
import numpy as np
loans  = pd.read_csv('loans.csv')

C:\Users\SPN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
loans_df = loans.copy()
loans_df.shape

(565176, 53)

In [3]:
loans_df.head()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,hardship_flag,disbursement_method,debt_settlement_flag
0,0,98717281,7000.0,7000.0,7000.0,36 months,14.99,242.63,C,C4,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
1,1,98825679,4800.0,4800.0,4800.0,36 months,13.99,164.03,C,C3,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
2,2,99225641,6000.0,6000.0,6000.0,36 months,8.24,188.69,B,B1,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
3,3,98897686,12000.0,12000.0,12000.0,36 months,12.74,402.83,C,C1,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
4,4,96823466,13500.0,13500.0,13500.0,60 months,14.99,321.10,C,C4,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N


In [4]:
#drop the first column since it replicates the index column
loans_df.drop('Unnamed: 0',inplace=True,axis=1)

In [5]:
loans_df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,home_ownership,...,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,hardship_flag,disbursement_method,debt_settlement_flag
0,98717281,7000.0,7000.0,7000.0,36 months,14.99,242.63,C,C4,RENT,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
1,98825679,4800.0,4800.0,4800.0,36 months,13.99,164.03,C,C3,OWN,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
2,99225641,6000.0,6000.0,6000.0,36 months,8.24,188.69,B,B1,RENT,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
3,98897686,12000.0,12000.0,12000.0,36 months,12.74,402.83,C,C1,RENT,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N
4,96823466,13500.0,13500.0,13500.0,60 months,14.99,321.10,C,C4,RENT,...,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,Cash,N


We have reduced the size of the dataset to make it easier to work with, by:

    removing the desc column:
        which contains a long text explanation for each loan
    removing the url column:
        which contains a link to each loan on Lending Club which can only be accessed with an investor account
    removing all columns containing more than 50% missing values:
        which allows us to move faster since we can spend less time trying to fill these values


In [6]:
loans_df.columns

Index(['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'purpose', 'zip_code', 'addr_state', 'delinq_2yrs', 'earliest_cr_line',
       'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt', 'last_credit_pull_d',
       'last_fico_range_high', 'last_fico_range_low',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens', 'hardship_flag',
       'disbursement_method', 'debt_settlement_flag'],
      dtype='object

After analyzing each column, we can conclude that the following features need to be removed:

- id: randomly generated field by Lending Club for unique identification purposes only
- member_id: also a randomly generated field by Lending Club for unique identification purposes only
- funded_amnt: leaks data from the future (after the loan is already started to be funded)
- funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
- grade: contains redundant information as the interest rate column (int_rate)
- sub_grade: also contains redundant information as the interest rate column (int_rate)
- emp_title: requires other data and a lot of processing to potentially be useful
- issue_d: leaks data from the future (after the loan is already completely funded)
- zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
- out_prncp: leaks data from the future, (after the loan already started to be paid off)
- out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)
- total_rec_int: leaks data from the future, (after the loan already started to be paid off),
- total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
- recoveries: also leaks data from the future, (after the loan already started to be paid off),
- collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).


Recall that Lending Club assigns a grade and a sub-grade based on the borrower's interest rate. While the grade and sub_grade values are categorical, the int_rate column contains continuous values, which are better suited for machine learning.

The out_prncp and out_prncp_inv both describe the outstanding principal amount for a loan, which is the remaining amount the borrower still owes. These 2 columns as well as the total_pymnt column describe properties of the loan after it's fully funded and started to be paid off. This information isn't available to an investor before the loan is fully funded and we don't want to include it in our model.

Moreover, some of these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower.

Let's now drop these columns from the Dataframe before moving on.

In [7]:
loans_df.shape

(565176, 52)

In [8]:
columns_to_drop =["id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade",
                  "issue_d","total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", 
                  "last_pymnt_amnt","zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", 
                  "total_pymnt_inv","total_rec_prncp"]

loans_df = loans_df.drop(columns_to_drop, axis=1)

**We now need to decide on a target column that we want to use for modeling.**

We should use the **loan_status** column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model. Let's explore the different values in this column and come up with a strategy for converting the values in this column.

In [9]:
print(loans_df['loan_status'].value_counts())

Fully Paid            295609
Current               187538
Charged Off            72921
Late (31-120 days)      5817
In Grace Period         2139
Late (16-30 days)       1128
Default                   14
Name: loan_status, dtype: int64


There are 8 different possible values for the **loan_status** column. You can read about the different loan statuses [here](https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-) and [here](https://forum.lendacademy.com/?topic=2427.msg20813#msg20813)

Only the Fully Paid and Charged Off values describe the final outcome of the loan.

In [10]:
loans_df.shape

(565176, 35)

## Binary Classification

In [11]:
#Remove all rows from loans_2007 that contain values other than Fully Paid or Charged Off 
#for the loan_status column.

loans_df = loans_df[(loans_df['loan_status'] == "Fully Paid") | (loans_df['loan_status'] == "Charged Off")]

#Use the Dataframe method replace to replace:
#Fully Paid with 1
#Charged Off with 0

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_df = loans_df.replace(status_replace)

#### To wrap up this mission, let's look for any columns that contain only one unique value and remove them


In [12]:
orig_columns = loans_df.columns
drop_columns = []
#For each column: Use the Series method dropna to remove any null values 
#and then use the Series method unique to return the set of non-null unique values
for col in orig_columns:
    col_series = loans_df[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
        
loans_df = loans_df.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'policy_code', 'hardship_flag']


In [13]:
loans_df.isnull().sum()

loan_amnt                      0
term                           0
int_rate                       0
installment                    0
home_ownership                 0
annual_inc                     0
verification_status            0
loan_status                    0
purpose                        0
addr_state                     0
delinq_2yrs                    0
earliest_cr_line               0
fico_range_low                 0
fico_range_high                0
inq_last_6mths                 0
open_acc                       0
pub_rec                        0
revol_bal                      0
total_acc                      0
initial_list_status            0
last_credit_pull_d            16
last_fico_range_high           0
last_fico_range_low            0
collections_12_mths_ex_med     0
application_type               0
acc_now_delinq                 0
chargeoff_within_12_mths       0
delinq_amnt                    0
pub_rec_bankruptcies           0
tax_liens                      0
disburseme

## Preparing Features

In [14]:
loans_df = loans_df.drop('last_credit_pull_d', axis=1)

In [15]:
loans_df.shape

(368530, 31)

#### Analyzing value counts for each column

In [16]:
loans_df['pub_rec_bankruptcies'].value_counts()

0.0    325448
1.0     41096
2.0      1546
3.0       318
4.0        80
5.0        30
6.0         7
8.0         3
7.0         2
Name: pub_rec_bankruptcies, dtype: int64

In [17]:
# very little variability in the column so we shall drop the column
loans_df = loans_df.drop('pub_rec_bankruptcies', axis=1)

In [18]:
loans_df.dtypes

loan_amnt                     float64
term                           object
int_rate                      float64
installment                   float64
home_ownership                 object
annual_inc                    float64
verification_status            object
loan_status                     int64
purpose                        object
addr_state                     object
delinq_2yrs                   float64
earliest_cr_line               object
fico_range_low                float64
fico_range_high               float64
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
total_acc                     float64
initial_list_status            object
last_fico_range_high          float64
last_fico_range_low           float64
collections_12_mths_ex_med    float64
application_type               object
acc_now_delinq                float64
chargeoff_within_12_mths      float64
delinq_amnt 

In [19]:
loans_df.corr()

,loan_amnt,int_rate,installment,annual_inc,loan_status,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,tax_liens
loan_amnt,1.000000,0.187868,0.956255,0.311738,-0.067565,0.002265,0.084101,0.084100,0.013056,0.179562,-0.048981,0.305596,0.209710,0.037112,0.038617,-0.014229,0.004990,0.001399,0.001554,0.010175
int_rate,0.187868,1.000000,0.202702,-0.053942,-0.230792,0.057488,-0.442379,-0.442375,0.209456,0.005931,0.043256,-0.000900,-0.035863,-0.325267,-0.250668,0.014918,0.014650,0.014017,0.002097,0.014703
installment,0.956255,0.202702,1.000000,0.300517,-0.063272,0.011185,0.026762,0.026761,0.033046,0.173325,-0.039596,0.294436,0.189581,0.016797,0.022384,-0.011585,0.006533,0.002259,0.001960,0.015661
annual_inc,0.311738,-0.053942,0.300517,1.000000,0.040367,0.045566,0.067524,0.067524,0.048876,0.142582,-0.001851,0.296174,0.189598,0.063849,0.057799,0.000740,0.013121,0.009848,0.008283,0.036696
loan_status,-0.067565,-0.230792,-0.063272,0.040367,1.000000,-0.023820,0.125998,0.125997,-0.052942,-0.021529,-0.030519,0.016013,0.024804,0.650617,0.560462,-0.021288,-0.006918,-0.005358,-0.001892,-0.017788
delinq_2yrs,0.002265,0.057488,0.011185,0.045566,-0.023820,1.000000,-0.167436,-0.167434,0.017018,0.055784,-0.025174,-0.026592,0.121245,-0.067111,-0.032517,0.070182,0.116441,0.134796,0.030171,0.007234
fico_range_low,0.084101,-0.442379,0.026762,0.067524,0.125998,-0.167436,1.000000,1.000000,-0.058808,0.014313,-0.170161,0.012953,0.034087,0.331124,0.238976,-0.058713,-0.034086,-0.047833,-0.012301,-0.047633
fico_range_high,0.084100,-0.442375,0.026761,0.067524,0.125997,-0.167434,1.000000,1.000000,-0.058809,0.014313,-0.170159,0.012952,0.034087,0.331124,0.238976,-0.058713,-0.034084,-0.047831,-0.012300,-0.047633
inq_last_6mths,0.013056,0.209456,0.033046,0.048876,-0.052942,0.017018,-0.058808,-0.058809,1.000000,0.136988,0.025336,0.009893,0.156285,-0.081080,-0.059338,0.003023,-0.005105,0.007238,-0.001528,0.003044
open_acc,0.179562,0.005931,0.173325,0.142582,-0.021529,0.055784,0.014313,0.014313,0.136988,1.000000,-0.009772,0.221212,0.691633,0.011873,0.027844,0.010853,0.020243,0.008600,0.006431,0.001119


In [20]:
## correlation with target column
loans_df.corr()['loan_status']

loan_amnt                    -0.067565
int_rate                     -0.230792
installment                  -0.063272
annual_inc                    0.040367
loan_status                   1.000000
delinq_2yrs                  -0.023820
fico_range_low                0.125998
fico_range_high               0.125997
inq_last_6mths               -0.052942
open_acc                     -0.021529
pub_rec                      -0.030519
revol_bal                     0.016013
total_acc                     0.024804
last_fico_range_high          0.650617
last_fico_range_low           0.560462
collections_12_mths_ex_med   -0.021288
acc_now_delinq               -0.006918
chargeoff_within_12_mths     -0.005358
delinq_amnt                  -0.001892
tax_liens                    -0.017788
Name: loan_status, dtype: float64

In [21]:
#drop columns with very low correlation with the target column
low_corr_columns = ['acc_now_delinq','chargeoff_within_12_mths','delinq_amnt']
loans_df = loans_df.drop(low_corr_columns, axis=1)

## Collinearity

In [22]:
#instalment and loan_amt, fico_range_low and fico_range_high
#this combination of columns are collinear so we shall drop one of each
loans_df = loans_df.drop(['fico_range_low','installment'], axis=1)

In [23]:
loans_df.shape

(368530, 25)

In [24]:
loans_numeric = loans_df.select_dtypes(include=["float64",'int64'])

In [25]:
loans_numeric.corr()

,loan_amnt,int_rate,annual_inc,loan_status,delinq_2yrs,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,tax_liens
loan_amnt,1.000000,0.187868,0.311738,-0.067565,0.002265,0.084100,0.013056,0.179562,-0.048981,0.305596,0.209710,0.037112,0.038617,-0.014229,0.010175
int_rate,0.187868,1.000000,-0.053942,-0.230792,0.057488,-0.442375,0.209456,0.005931,0.043256,-0.000900,-0.035863,-0.325267,-0.250668,0.014918,0.014703
annual_inc,0.311738,-0.053942,1.000000,0.040367,0.045566,0.067524,0.048876,0.142582,-0.001851,0.296174,0.189598,0.063849,0.057799,0.000740,0.036696
loan_status,-0.067565,-0.230792,0.040367,1.000000,-0.023820,0.125997,-0.052942,-0.021529,-0.030519,0.016013,0.024804,0.650617,0.560462,-0.021288,-0.017788
delinq_2yrs,0.002265,0.057488,0.045566,-0.023820,1.000000,-0.167434,0.017018,0.055784,-0.025174,-0.026592,0.121245,-0.067111,-0.032517,0.070182,0.007234
fico_range_high,0.084100,-0.442375,0.067524,0.125997,-0.167434,1.000000,-0.058809,0.014313,-0.170159,0.012952,0.034087,0.331124,0.238976,-0.058713,-0.047633
inq_last_6mths,0.013056,0.209456,0.048876,-0.052942,0.017018,-0.058809,1.000000,0.136988,0.025336,0.009893,0.156285,-0.081080,-0.059338,0.003023,0.003044
open_acc,0.179562,0.005931,0.142582,-0.021529,0.055784,0.014313,0.136988,1.000000,-0.009772,0.221212,0.691633,0.011873,0.027844,0.010853,0.001119
pub_rec,-0.048981,0.043256,-0.001851,-0.030519,-0.025174,-0.170159,0.025336,-0.009772,1.000000,-0.082275,-0.010914,-0.052417,-0.043539,0.021352,0.691766
revol_bal,0.305596,-0.000900,0.296174,0.016013,-0.026592,0.012952,0.009893,0.221212,-0.082275,1.000000,0.202757,0.050503,0.049978,-0.020167,-0.007251


## Text Columns

In [26]:
loans_object = loans_df.select_dtypes(include=["object"])
loans_object.columns.to_list()

['term',
 'home_ownership',
 'verification_status',
 'purpose',
 'addr_state',
 'earliest_cr_line',
 'initial_list_status',
 'application_type',
 'disbursement_method',
 'debt_settlement_flag']

In [27]:
cols = ['home_ownership' , 'verification_status', 'term', 'addr_state']
for c in cols:
    print(loans_df[c].value_counts())

MORTGAGE    185446
RENT        145312
OWN          37430
ANY            254
OTHER           46
NONE            42
Name: home_ownership, dtype: int64
Verified           141714
Source Verified    115798
Not Verified       111018
Name: verification_status, dtype: int64
 36 months    282974
 60 months     85556
Name: term, dtype: int64
CA    56443
NY    30425
TX    29992
FL    26366
IL    13882
NJ    13388
PA    12316
OH    11655
GA    11631
NC    10517
VA    10434
MI     9337
AZ     9068
MD     8527
MA     8481
WA     8190
CO     8152
MN     6401
MO     5798
NV     5722
CT     5494
IN     5420
TN     5003
WI     4711
OR     4637
AL     4533
SC     4296
LA     4224
KY     3435
OK     3317
KS     3174
UT     2868
AR     2794
HI     2030
NM     1888
NH     1742
RI     1623
MS     1077
MT     1057
WV     1009
DE     1006
DC      998
AK      905
NE      901
WY      818
SD      751
VT      650
ID      562
ME      513
ND      368
IA        1
Name: addr_state, dtype: int64


The home_ownership, verification_status, term, and addr_state columns all contain multiple discrete values.

In [28]:
loans_df.shape

(368530, 25)

In [29]:
print(loans_df["term"].value_counts())
print(loans_df["purpose"].value_counts())

 36 months    282974
 60 months     85556
Name: term, dtype: int64
debt_consolidation    215817
credit_card            77026
home_improvement       23429
other                  22314
major_purchase          8081
small_business          4758
medical                 4099
car                     4059
moving                  2551
vacation                2424
house                   2375
wedding                 1332
renewable_energy         265
Name: purpose, dtype: int64


The home_ownership, verification_status, and term columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.

In [30]:
loans_df = loans_df.drop(['addr_state', 'earliest_cr_line'], axis=1)

In [31]:
loans_object = loans_df.select_dtypes(include=["object"])

In [32]:
loans_df[loans_object.columns.to_list()].head()

,term,home_ownership,verification_status,purpose,initial_list_status,application_type,disbursement_method,debt_settlement_flag
1,36 months,OWN,Source Verified,credit_card,w,Individual,Cash,N
2,36 months,RENT,Source Verified,credit_card,w,Individual,Cash,N
3,36 months,RENT,Source Verified,debt_consolidation,f,Individual,Cash,N
5,60 months,OWN,Verified,debt_consolidation,w,Individual,Cash,N
6,36 months,MORTGAGE,Source Verified,debt_consolidation,w,Individual,Cash,N


In [33]:
cols = ['initial_list_status',
 'application_type',
 'disbursement_method',
 'debt_settlement_flag']

for c in cols:
    print(loans_df[c].value_counts())

f    198356
w    170174
Name: initial_list_status, dtype: int64
Individual    360699
Joint App       7831
Name: application_type, dtype: int64
Cash         367686
DirectPay       844
Name: disbursement_method, dtype: int64
N    360494
Y      8036
Name: debt_settlement_flag, dtype: int64


In [34]:
#we will now convert the categorical  columns into numeric by with the help of the **get_dummies* function
# the we shall drop the categorical columns entirely
cat_columns = ['term',
 'home_ownership',
 'verification_status',
 'purpose',
 'initial_list_status',
 'application_type',
 'disbursement_method',
 'debt_settlement_flag']
dummy_df = pd.get_dummies(loans_df[cat_columns])
loans_df = pd.concat([loans_df, dummy_df], axis=1)
loans_df = loans_df.drop(cat_columns, axis=1)

In [35]:
loans_df.head()

,loan_amnt,int_rate,annual_inc,loan_status,delinq_2yrs,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,...,purpose_vacation,purpose_wedding,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,disbursement_method_Cash,disbursement_method_DirectPay,debt_settlement_flag_N,debt_settlement_flag_Y
1,4800.0,13.99,35000.0,1,0.0,674.0,2.0,5.0,0.0,3120.0,...,0,0,0,1,1,0,1,0,1,0
2,6000.0,8.24,40200.0,1,0.0,759.0,0.0,25.0,0.0,3875.0,...,0,0,0,1,1,0,1,0,1,0
3,12000.0,12.74,130000.0,1,0.0,664.0,0.0,6.0,0.0,43032.0,...,0,0,1,0,1,0,1,0,1,0
5,33775.0,13.49,155000.0,1,1.0,714.0,0.0,16.0,0.0,5559.0,...,0,0,0,1,1,0,1,0,1,0
6,20000.0,11.49,100000.0,1,0.0,709.0,2.0,12.0,0.0,17789.0,...,0,0,0,1,1,0,1,0,1,0


In [36]:
loans_df.to_csv('cleaned_loans.csv')

We converted all of the columns to numerical values because those are the only type of value scikit-learn can work with. Now let's experiment with training models and evaluating accuracy using cross-validation.

## Training And Predicting

#### Picking an error metric

Let's use **accuracy** as our metric

A good first algorithm to apply to binary classification problems is logistic regression, for the following reasons:

- it's quick to train and we can iterate more quickly,
- it's less prone to overfitting than more complex models like decision trees,
- it's easy to interpret.


We will also employ this algorithm to our problem.

In [37]:
cleaned_loans = pd.read_csv('cleaned_loans.csv')

In [38]:
from sklearn.linear_model import LogisticRegression
#Create a Dataframe named features that contains just the feature columns.
#Remove the loan_status column.
lr = LogisticRegression()
cols = cleaned_loans.columns
train_cols = cols.drop("loan_status")
features = cleaned_loans[train_cols]
target = cleaned_loans["loan_status"]
lr.fit(features, target)
predictions = lr.predict(features)

C:\Users\SPN\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In order to get a realistic depiction of the accuracy of the model, we will perform k-fold cross validation. We can use the **cross_val_predict()** function from the **sklearn.model_selection** package.

In [39]:
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression()
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)
# False positives.
fp_filter = (predictions == 1) & (cleaned_loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (cleaned_loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (cleaned_loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (cleaned_loans["loan_status"] == 0)
# True negatives
tn_filter = (predictions == 0) & (cleaned_loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)


C:\Users\SPN\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\SPN\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regress

0.9460943340696664
0.36097969034983063


C:\Users\SPN\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [40]:
#check for class imbalance in the loan_status column
class_filter = (cleaned_loans["loan_status"] == 1)
ones = cleaned_loans[class_filter].shape[0]
number_ones = ones/cleaned_loans.shape[0]
number_ones

0.8021300843893305

We notice that there is a significant class imbalance in the loan_status column. About 80% of loans were paid off on time (1), but only about 20% of loans weren't paid off on time (0). This causes a major issue when we use accuracy as a metric. This is due to the class imbalance, a classifier can predict 1 for every row, and still have high accuracy.

Let's now repeat the cross validation procedure we performed, but with the class_weight parameter set to balanced.

In [41]:
lr = LogisticRegression(class_weight="balanced")
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (cleaned_loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (cleaned_loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (cleaned_loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (cleaned_loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

C:\Users\SPN\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8484010973955461
0.14294921901784122


We notice that the false positive rate has lowered from around 0.36 to about 0.14- by using cross validation.

Let's try to lower the false positive rate further by assigning a harsher penalty for misclassifying the negative class. While setting class_weight to balanced will automatically set a penalty based on the number of 1s and 0s in the column, we can also set a manual penalty. Above, the penalty scikit-learn imposed for misclassifying a 0 would have been around 6 (since there are 6 times as many 1s as 0s).

We can also specify a penalty manually if we want to adjust the rates more. To do this, we need to pass in a dictionary of penalty values to the class_weight parameter:

In [42]:
#Modify the code from the last screen to change the class_weight parameter 
#from the string "balanced" to the dictionary as below
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight=penalty)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.

fp_filter = (predictions == 1) & (cleaned_loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (cleaned_loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (cleaned_loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (cleaned_loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

C:\Users\SPN\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7388814278320349
0.07513610619711743


It looks like assigning manual penalties lowered the false positive rate to about 8%, and thus lowered our risk. Note that this comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. Given that we're approaching this as a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs.

Now, let's try a more complex algorithm, random forest.

Random forests are able to work with nonlinear data, and learn complex conditionals. Logistic regressions are only able to work with linear data. Training a random forest algorithm may enable us to get more accuracy due to columns that correlate nonlinearly with loan_status.

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

rf = RandomForestClassifier(class_weight="balanced", random_state=1)
predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (cleaned_loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.`
tp_filter = (predictions == 1) & (cleaned_loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (cleaned_loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (cleaned_loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.623353822109611
0.46048463405603324


Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s

Ultimately, our best model had a false positive rate of nearly 8%, and a true positive rate of nearly 73%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 8% of borrowers defaulting, and that the pool of 73% of borrowers is large enough to make enough interest money to offset the losses.

## Recommendations

There's still quite a bit of room to improve:
- We can tweak the penalties further.
- We can try models other than a random forest and logistic regression.
- We can use some of the columns we discarded to generate better features.
- We can ensemble multiple models to get more accurate predictions.
- We can tune the parameters of the algorithm to achieve higher performance.